In [23]:
#This program finds the lag between Advertising and Sales spend vs realized Sales
# The lag is determined using simple OLS/linear regression


import pandas as pd
import statsmodels.api as sm


file_path = 'SaltPath.xlsx'  
df = pd.read_excel(file_path)

# Drop Q4 for multicollinearity
df = df.drop(columns=['Q4'])

# Create a function to fit models with different lags and return the best one based on R-squared
def fit_best_model(df):
    best_r2 = -float('inf')
    best_model = None
    best_lag = 0

    for lag in range(1, 5):  # Test lags from 1 to 4 quarters
        df[f'Adv_Lag{lag}'] = df['Adv'].shift(lag)
        df[f'Prom_Lag{lag}'] = df['Prom'].shift(lag)
        df_lagged = df.dropna()

        Y = df_lagged['Sales']
        X = df_lagged[['Prom', 'Adv', 'Q2', f'Adv_Lag{lag}', f'Prom_Lag{lag}']]
        X = sm.add_constant(X)

        model = sm.OLS(Y, X).fit()
        if model.rsquared > best_r2:
            best_r2 = model.rsquared
            best_model = model
            best_lag = lag

    return best_model, best_lag

print('Lag added for ',best_lag, 'quarters')

# Fit the best model
best_model, best_lag = fit_best_model(df)

print(best_model.summary())


Lag added for  1 quarters
                            OLS Regression Results                            
Dep. Variable:                  Sales   R-squared:                       0.873
Model:                            OLS   Adj. R-squared:                  0.861
Method:                 Least Squares   F-statistic:                     72.69
Date:                Wed, 20 Nov 2024   Prob (F-statistic):           1.70e-22
Time:                        18:59:41   Log-Likelihood:                -266.43
No. Observations:                  59   AIC:                             544.9
Df Residuals:                      53   BIC:                             557.3
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const        166.7028     